# Phase 1 : loading and basic cleaning 

#### Important Libs

In [8]:
import numpy as np
import pandas as pd
from yaml import safe_load
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

#### getting our yaml file/match files 

In [11]:
filenames = []
for file in os.listdir('data'):
    if file.endswith('.yaml'):
        filenames.append(os.path.join('data',file))

In [12]:
filenames[0:5]

['data/1394760.yaml',
 'data/1299579.yaml',
 'data/1420225.yaml',
 'data/406198.yaml',
 'data/951405.yaml']

In [ ]:
final_df = pd.DataFrame()
counter = 1
for file in tqdm(filenames):
    with open(file, 'r') as f:
        df = pd.json_normalize(safe_load(f))
        df['match_id'] = counter
        final_df = pd.concat([final_df,df])
        counter+=1
        
final_df

  0%|          | 0/3857 [00:00<?, ?it/s]

  9%|▉         | 363/3857 [03:25<48:35,  1.20it/s]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f166838b7d0>>
Traceback (most recent call last):
  File "/home/manan/miniconda3/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
 10%|▉         | 382/3857 [03:41<44:55,  1.29it/s]

In [4]:
final_df

NameError: name 'final_df' is not defined

In [3]:
backup = final_df.copy()

NameError: name 'final_df' is not defined

In [2]:
backup

NameError: name 'backup' is not defined

#### dropping unwanted column

In [8]:
final_df.drop(columns=[
    'meta.data_version',
    'meta.created',
    'meta.revision',
    'info.outcome.bowl_out',
    'info.bowl_out',
    'info.supersubs.South Africa',
    'info.supersubs.New Zealand',
    'info.outcome.eliminator',
    'info.outcome.result',
    'info.outcome.method',
    'info.neutral_venue',
    'info.match_type_number',
    'info.outcome.by.runs',
    'info.outcome.by.wickets'
],inplace=True)

NameError: name 'final_df' is not defined

In [9]:
final_df['info.gender'].value_counts()


NameError: name 'final_df' is not defined

- info.gender
- male      966
- female    466
- Name: count, dtype: int64

In [11]:
final_df = final_df[final_df['info.gender'] == 'male']


In [12]:
final_df.drop(columns=['info.gender'],inplace=True)


In [1]:
final_df

NameError: name 'final_df' is not defined

In [7]:
final_df['info.match_type'].value_counts()


NameError: name 'final_df' is not defined

- info.match_type
- T20    966
- Name: count, dtype: int64

In [6]:
final_df['info.overs'].value_counts()



NameError: name 'final_df' is not defined

In [5]:
final_df = final_df[final_df['info.overs'] == 20]
final_df.drop(columns=['info.overs','info.match_type'],inplace=True)
final_df

NameError: name 'final_df' is not defined

In [20]:
final_df['info.venue']	

0                   Melbourne Cricket Ground
0             Simonds Stadium, South Geelong
0                              Adelaide Oval
0          Bready Cricket Club, Magheramason
0                         Harare Sports Club
                      ...                   
0              Shere Bangla National Stadium
0              Shere Bangla National Stadium
0                                ICC Academy
0    Pallekele International Cricket Stadium
0                        R Premadasa Stadium
Name: info.venue, Length: 2496, dtype: object

In [1]:
import pickle
pickle.dump(final_df,open('dataset_level1.pkl','wb'))


NameError: name 'final_df' is not defined

# Phase 2 Cleaning data tha actually we need

### So After cleaning data i dump all things in pickel so i dont need to do the whole thing of loading ymal files again to final df 

In [9]:
import pickle

with open('dataset_level1.pkl', 'rb') as f:
    final_df = pickle.load(f)

In [11]:
matches = pickle.load(open('dataset_level1.pkl','rb'))


In [13]:
#matches = pickle.load(open('dataset_level1.pkl','rb'))
matches.iloc[0]['innings'][0]['1st innings']['deliveries']

[{0.1: {'batsman': 'AJ Finch',
   'bowler': 'SL Malinga',
   'non_striker': 'M Klinger',
   'runs': {'batsman': 0, 'extras': 0, 'total': 0}}},
 {0.2: {'batsman': 'AJ Finch',
   'bowler': 'SL Malinga',
   'non_striker': 'M Klinger',
   'runs': {'batsman': 0, 'extras': 0, 'total': 0}}},
 {0.3: {'batsman': 'AJ Finch',
   'bowler': 'SL Malinga',
   'non_striker': 'M Klinger',
   'runs': {'batsman': 1, 'extras': 0, 'total': 1}}},
 {0.4: {'batsman': 'M Klinger',
   'bowler': 'SL Malinga',
   'non_striker': 'AJ Finch',
   'runs': {'batsman': 2, 'extras': 0, 'total': 2}}},
 {0.5: {'batsman': 'M Klinger',
   'bowler': 'SL Malinga',
   'non_striker': 'AJ Finch',
   'runs': {'batsman': 0, 'extras': 0, 'total': 0}}},
 {0.6: {'batsman': 'M Klinger',
   'bowler': 'SL Malinga',
   'non_striker': 'AJ Finch',
   'runs': {'batsman': 3, 'extras': 0, 'total': 3}}},
 {1.1: {'batsman': 'M Klinger',
   'bowler': 'KMDN Kulasekara',
   'non_striker': 'AJ Finch',
   'runs': {'batsman': 0, 'extras': 0, 'total': 

In [9]:
count = 1
delivery_df = pd.DataFrame()
for index, row in matches.iterrows():
    if count in [75,108,150,180,268,360,443,458,584,748,982,1052,1111,1226,1345]:
        count+=1
        continue
    count+=1
    ball_of_match = []
    batsman = []
    bowler = []
    runs = []
    player_of_dismissed = []
    teams = []
    batting_team = []
    match_id = []
    city = []
    venue = []
    for ball in row['innings'][0]['1st innings']['deliveries']:
        for key in ball.keys():
            match_id.append(count)
            batting_team.append(row['innings'][0]['1st innings']['team'])
            teams.append(row['info.teams'])
            ball_of_match.append(key)
            batsman.append(ball[key]['batsman'])
            bowler.append(ball[key]['bowler'])
            runs.append(ball[key]['runs']['total'])
            city.append(row['info.city'])
            venue.append(row['info.venue'])
            try:
                player_of_dismissed.append(ball[key]['wicket']['player_out'])
            except:
                player_of_dismissed.append('0')
    loop_df = pd.DataFrame({
            'match_id':match_id,
            'teams':teams,
            'batting_team':batting_team,
            'ball':ball_of_match,
            'batsman':batsman,
            'bowler':bowler,
            'runs':runs,
            'player_dismissed':player_of_dismissed,
            'city':city,
            'venue':venue
        })
    delivery_df = pd.concat([delivery_df,loop_df])

In [10]:
delivery_df


,match_id,teams,batting_team,ball,batsman,bowler,runs,player_dismissed,city,venue
0,2,"[Australia, Sri Lanka]",Australia,0.1,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground
1,2,"[Australia, Sri Lanka]",Australia,0.2,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground
2,2,"[Australia, Sri Lanka]",Australia,0.3,AJ Finch,SL Malinga,1,0,NaN,Melbourne Cricket Ground
3,2,"[Australia, Sri Lanka]",Australia,0.4,M Klinger,SL Malinga,2,0,NaN,Melbourne Cricket Ground
4,2,"[Australia, Sri Lanka]",Australia,0.5,M Klinger,SL Malinga,0,0,NaN,Melbourne Cricket Ground
...,...,...,...,...,...,...,...,...,...,...
121,2497,"[Sri Lanka, Australia]",Sri Lanka,19.3,SMSM Senanayake,MA Starc,1,0,Colombo,R Premadasa Stadium
122,2497,"[Sri Lanka, Australia]",Sri Lanka,19.4,DM de Silva,MA Starc,0,0,Colombo,R Premadasa Stadium
123,2497,"[Sri Lanka, Australia]",Sri Lanka,19.5,DM de Silva,MA Starc,0,DM de Silva,Colombo,R Premadasa Stadium
124,2497,"[Sri Lanka, Australia]",Sri Lanka,19.6,SMSM Senanayake,MA Starc,2,0,Colombo,R Premadasa Stadium


#### Adding bolwing_team column

In [13]:
def bowl(row):
    for team in row['teams']:
        if team != row['batting_team']:
            return team

In [14]:
delivery_df['bowling_team'] = delivery_df.apply(bowl,axis=1)


In [15]:
delivery_df


,match_id,teams,batting_team,ball,batsman,bowler,runs,player_dismissed,city,venue,bowling_team
0,2,"[Australia, Sri Lanka]",Australia,0.1,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground,Sri Lanka
1,2,"[Australia, Sri Lanka]",Australia,0.2,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground,Sri Lanka
2,2,"[Australia, Sri Lanka]",Australia,0.3,AJ Finch,SL Malinga,1,0,NaN,Melbourne Cricket Ground,Sri Lanka
3,2,"[Australia, Sri Lanka]",Australia,0.4,M Klinger,SL Malinga,2,0,NaN,Melbourne Cricket Ground,Sri Lanka
4,2,"[Australia, Sri Lanka]",Australia,0.5,M Klinger,SL Malinga,0,0,NaN,Melbourne Cricket Ground,Sri Lanka
...,...,...,...,...,...,...,...,...,...,...,...
121,2497,"[Sri Lanka, Australia]",Sri Lanka,19.3,SMSM Senanayake,MA Starc,1,0,Colombo,R Premadasa Stadium,Australia
122,2497,"[Sri Lanka, Australia]",Sri Lanka,19.4,DM de Silva,MA Starc,0,0,Colombo,R Premadasa Stadium,Australia
123,2497,"[Sri Lanka, Australia]",Sri Lanka,19.5,DM de Silva,MA Starc,0,DM de Silva,Colombo,R Premadasa Stadium,Australia
124,2497,"[Sri Lanka, Australia]",Sri Lanka,19.6,SMSM Senanayake,MA Starc,2,0,Colombo,R Premadasa Stadium,Australia


In [16]:
delivery_df.drop(columns=['teams'],inplace=True)


In [17]:
delivery_df['batting_team'].unique()


array(['Australia', 'Hong Kong', 'Zimbabwe', 'India', 'Bangladesh',
       'New Zealand', 'South Africa', 'England', 'West Indies',
       'Pakistan', 'Scotland', 'Oman', 'Ireland', 'Papua New Guinea',
       'United Arab Emirates', 'Sri Lanka', 'Netherlands', 'Nepal',
       'Vanuatu', 'Philippines', 'United States of America', 'Germany',
       'Ghana', 'Uganda', 'Kenya', 'Namibia', 'Nigeria', 'Botswana',
       'Guernsey', 'Denmark', 'Jersey', 'Italy', 'Norway', 'Thailand',
       'Malaysia', 'Maldives', 'Singapore', 'Kuwait', 'Bermuda', 'Canada',
       'Cayman Islands', 'Portugal', 'Gibraltar', 'Spain', 'Bhutan',
       'Qatar', 'Iran', 'Belgium', 'Isle of Man', 'Bulgaria', 'Romania',
       'Luxembourg', 'Austria', 'Czech Republic', 'Greece', 'Serbia',
       'Malta', 'France', 'Sweden', 'Rwanda', 'Finland', 'Hungary',
       'Estonia', 'Cyprus', 'Switzerland', 'Seychelles', 'Malawi',
       'Lesotho', 'Swaziland', 'Tanzania', 'Mozambique', 'Sierra Leone',
       'Cameroon', 'Bah

#### adding famous team only 

In [18]:
teams = [
    'Australia',
    'India',
    'Bangladesh',
    'New Zealand',
    'South Africa',
    'England',
    'West Indies',
    'Afghanistan',
    'Pakistan',
    'Sri Lanka'    
]

In [19]:
delivery_df = delivery_df[delivery_df['batting_team'].isin(teams)]
delivery_df = delivery_df[delivery_df['bowling_team'].isin(teams)]

In [20]:
delivery_df


,match_id,batting_team,ball,batsman,bowler,runs,player_dismissed,city,venue,bowling_team
0,2,Australia,0.1,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground,Sri Lanka
1,2,Australia,0.2,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground,Sri Lanka
2,2,Australia,0.3,AJ Finch,SL Malinga,1,0,NaN,Melbourne Cricket Ground,Sri Lanka
3,2,Australia,0.4,M Klinger,SL Malinga,2,0,NaN,Melbourne Cricket Ground,Sri Lanka
4,2,Australia,0.5,M Klinger,SL Malinga,0,0,NaN,Melbourne Cricket Ground,Sri Lanka
...,...,...,...,...,...,...,...,...,...,...
121,2497,Sri Lanka,19.3,SMSM Senanayake,MA Starc,1,0,Colombo,R Premadasa Stadium,Australia
122,2497,Sri Lanka,19.4,DM de Silva,MA Starc,0,0,Colombo,R Premadasa Stadium,Australia
123,2497,Sri Lanka,19.5,DM de Silva,MA Starc,0,DM de Silva,Colombo,R Premadasa Stadium,Australia
124,2497,Sri Lanka,19.6,SMSM Senanayake,MA Starc,2,0,Colombo,R Premadasa Stadium,Australia


### dataset_level2 where we have final columns that we needed 

In [21]:
output = delivery_df[['match_id','batting_team','bowling_team','ball','runs','player_dismissed','city','venue']]


In [22]:
output


,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,2,Australia,Sri Lanka,0.1,0,0,NaN,Melbourne Cricket Ground
1,2,Australia,Sri Lanka,0.2,0,0,NaN,Melbourne Cricket Ground
2,2,Australia,Sri Lanka,0.3,1,0,NaN,Melbourne Cricket Ground
3,2,Australia,Sri Lanka,0.4,2,0,NaN,Melbourne Cricket Ground
4,2,Australia,Sri Lanka,0.5,0,0,NaN,Melbourne Cricket Ground
...,...,...,...,...,...,...,...,...
121,2497,Sri Lanka,Australia,19.3,1,0,Colombo,R Premadasa Stadium
122,2497,Sri Lanka,Australia,19.4,0,0,Colombo,R Premadasa Stadium
123,2497,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,R Premadasa Stadium
124,2497,Sri Lanka,Australia,19.6,2,0,Colombo,R Premadasa Stadium


In [23]:
pickle.dump(output,open('dataset_level2.pkl','wb'))


In [24]:
output[output['city'].isnull()]['venue'].value_counts()


venue
Dubai International Cricket Stadium        3092
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sylhet International Cricket Stadium        128
Sharjah Cricket Stadium                     127
Carrara Oval                                 64
Name: count, dtype: int64

In [25]:
x = np.where(output['city'].isnull(), output['venue'].str.split().apply(lambda x:x[0]), output['city'])


In [26]:
output['city'] = x


In [29]:
output.isnull().sum()


match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
dtype: int64

In [34]:
#already removed so it get error 
output.drop(columns=['venue'],inplace=True)


KeyError: "['venue'] not found in axis"

In [31]:
output

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne
...,...,...,...,...,...,...,...
121,2497,Sri Lanka,Australia,19.3,1,0,Colombo
122,2497,Sri Lanka,Australia,19.4,0,0,Colombo
123,2497,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo
124,2497,Sri Lanka,Australia,19.6,2,0,Colombo


In [35]:
total_df = output.groupby('match_id').sum()['runs'].reset_index()
output = output.merge(total_df,on='match_id')

In [36]:
output


,match_id,batting_team,bowling_team,ball,runs_x,player_dismissed,city,runs_y
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,168
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,168
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,168
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,168
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,168
...,...,...,...,...,...,...,...,...
96708,2497,Sri Lanka,Australia,19.3,1,0,Colombo,128
96709,2497,Sri Lanka,Australia,19.4,0,0,Colombo,128
96710,2497,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,128
96711,2497,Sri Lanka,Australia,19.6,2,0,Colombo,128


In [37]:
output['current_score'] = output.groupby('match_id')['runs_x'].cumsum()


##### Extract Over & Ball Number : Extracts over and ball number from the ball column, assuming it's stored as over.ball (ex 5.2 means 5th over, 2nd ball).

In [38]:
output['over'] = output['ball'].apply(lambda x:str(x).split(".")[0])
output['ball_no'] = output['ball'].apply(lambda x:str(x).split(".")[1])

##### Compute Balls Bowled So  : Converts over number to balls bowled (each over has 6 balls)Ex 5.2 → (5*6) + 2 = 32 balls bowled.

In [39]:
output['balls_bowled'] = (output['over'].astype('int')*6) + output['ball_no'].astype('int')


In [40]:
output['crr'] = round((output['current_score']*6)/output['balls_bowled'],2)


In [41]:
output['player_dismissed'] = output['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
output['player_dismissed'] = output['player_dismissed'].astype('int')
output['player_dismissed'] = output.groupby('match_id')['player_dismissed'].cumsum()
output['wickets_left'] = 10 - output['player_dismissed']

#### final dataframe with all feature we need for model now let's clean some data we need  

In [42]:
final_df = output[['match_id','batting_team','bowling_team','runs_x','current_score','balls_bowled','wickets_left','crr','city','runs_y']]


In [43]:
final_df

,match_id,batting_team,bowling_team,runs_x,current_score,balls_bowled,wickets_left,crr,city,runs_y
0,2,Australia,Sri Lanka,0,0,1,10,0.00,Melbourne,168
1,2,Australia,Sri Lanka,0,0,2,10,0.00,Melbourne,168
2,2,Australia,Sri Lanka,1,1,3,10,2.00,Melbourne,168
3,2,Australia,Sri Lanka,2,3,4,10,4.50,Melbourne,168
4,2,Australia,Sri Lanka,0,3,5,10,3.60,Melbourne,168
...,...,...,...,...,...,...,...,...,...,...
96708,2497,Sri Lanka,Australia,1,125,117,2,6.41,Colombo,128
96709,2497,Sri Lanka,Australia,0,125,118,2,6.36,Colombo,128
96710,2497,Sri Lanka,Australia,0,125,119,1,6.30,Colombo,128
96711,2497,Sri Lanka,Australia,2,127,120,1,6.35,Colombo,128


In [44]:
final_df.drop(columns=['balls_bowled'],inplace=True)


In [45]:
groups = final_df.groupby('match_id')


In [46]:
match_ids = final_df['match_id'].unique()
last_five = []
for id in match_ids:
    last_five.extend(groups.get_group(id).rolling(window=30)['runs_x'].sum().values.tolist())

In [47]:
final_df['last_five'] = last_five


In [48]:
final_df


,match_id,batting_team,bowling_team,runs_x,current_score,wickets_left,crr,city,runs_y,last_five
0,2,Australia,Sri Lanka,0,0,10,0.00,Melbourne,168,NaN
1,2,Australia,Sri Lanka,0,0,10,0.00,Melbourne,168,NaN
2,2,Australia,Sri Lanka,1,1,10,2.00,Melbourne,168,NaN
3,2,Australia,Sri Lanka,2,3,10,4.50,Melbourne,168,NaN
4,2,Australia,Sri Lanka,0,3,10,3.60,Melbourne,168,NaN
...,...,...,...,...,...,...,...,...,...,...
96708,2497,Sri Lanka,Australia,1,125,2,6.41,Colombo,128,32.0
96709,2497,Sri Lanka,Australia,0,125,2,6.36,Colombo,128,32.0
96710,2497,Sri Lanka,Australia,0,125,1,6.30,Colombo,128,32.0
96711,2497,Sri Lanka,Australia,2,127,1,6.35,Colombo,128,33.0


In [49]:
final_df.dropna(inplace=True)


In [50]:
final_df


,match_id,batting_team,bowling_team,runs_x,current_score,wickets_left,crr,city,runs_y,last_five
29,2,Australia,Sri Lanka,1,43,10,8.60,Melbourne,168,43.0
30,2,Australia,Sri Lanka,1,44,10,8.52,Melbourne,168,44.0
31,2,Australia,Sri Lanka,1,45,10,8.44,Melbourne,168,45.0
32,2,Australia,Sri Lanka,0,45,10,8.18,Melbourne,168,44.0
33,2,Australia,Sri Lanka,0,45,10,7.94,Melbourne,168,42.0
...,...,...,...,...,...,...,...,...,...,...
96708,2497,Sri Lanka,Australia,1,125,2,6.41,Colombo,128,32.0
96709,2497,Sri Lanka,Australia,0,125,2,6.36,Colombo,128,32.0
96710,2497,Sri Lanka,Australia,0,125,1,6.30,Colombo,128,32.0
96711,2497,Sri Lanka,Australia,2,127,1,6.35,Colombo,128,33.0


In [51]:
eligible_cities = final_df['city'].value_counts()[final_df['city'].value_counts() > 600].index.tolist()
final_df = final_df[final_df['city'].isin(eligible_cities)]

In [ ]:
final_df

,match_id,batting_team,bowling_team,runs_x,current_score,wickets_left,crr,city,runs_y,last_five
29,2,Australia,Sri Lanka,1,43,10,8.60,Melbourne,168,43.0
30,2,Australia,Sri Lanka,1,44,10,8.52,Melbourne,168,44.0
31,2,Australia,Sri Lanka,1,45,10,8.44,Melbourne,168,45.0
32,2,Australia,Sri Lanka,0,45,10,8.18,Melbourne,168,44.0
33,2,Australia,Sri Lanka,0,45,10,7.94,Melbourne,168,42.0
...,...,...,...,...,...,...,...,...,...,...
96708,2497,Sri Lanka,Australia,1,125,2,6.41,Colombo,128,32.0
96709,2497,Sri Lanka,Australia,0,125,2,6.36,Colombo,128,32.0
96710,2497,Sri Lanka,Australia,0,125,1,6.30,Colombo,128,32.0
96711,2497,Sri Lanka,Australia,2,127,1,6.35,Colombo,128,33.0


: 